In [8]:
import os
import sys
import warnings

from glob import glob
import random
import datetime
import re
from tqdm import tqdm

import pandas as pd
import numpy as np

from google.cloud import bigquery

import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

import gspread
from oauth2client.service_account import ServiceAccountCredentials


from plotnine import *

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
font = fm.FontProperties(fname=path, size=9)

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', -1)
pd.options.display.float_format = '{:.2f}'.format

In [7]:
def bigquery_to_pandas(query_string) :

    credential_path = '../99_credential/lawtalk-bigquery-2bfd97cf0729.json'
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path
    bqclient = bigquery.Client()
    
    b = (
        bqclient.query(query_string)
        .result()
        .to_dataframe(
            # Optionally, explicitly request to use the BigQuery Storage API. As of
            # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
            # API is used by default.
            create_bqstorage_client=True,
        )
    )
    
    return b

In [ ]:
def gs_append(url, credential_path = '../99_credential/lawtalk-bigquery-2bfd97cf0729.json', scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']) : 
    
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credential_path, scope)
    
    gc = gspread.authorize(credentials)
    doc = gc.open_by_url(url)
    
    sheet_name = re.search(string = str(doc.get_worksheet(0)), pattern = r"\'(.*)\'").group(0)[1:-1]
    sheet = doc.worksheet(sheet_name)
    
    sheet_content = sheet.get_all_values()
    
    df_ = pd.DataFrame(sheet_content[1:], columns = sheet_content[0])
    
    return df_

In [1]:
def read_bql(file_name) :
    with open(file_name, "r") as f :
        bql = f.read()
    return bql

In [10]:
bigquery_to_pandas(read_bql("../bql/advicetransaction.bql"))

,_id,no_by_advice,number,createdAt,updatedAt,requestedAt,advice,user,lawyer_id,seller,...,coupon,metadata,metadata_page,metadata_source,metadata_element,metadata_position,metadata_context,metadata_contextAdditional,metadata_extraInfo,slug
0,62cc9d4f506f4dec562448ed,1,NaN,2022-07-12 06:59:43.413,2022-07-12 07:00:26.442,2022-07-12 06:59:43,62cc9d0c14e3a4029ab9bd29,62cc9ceaff6355002fe28468,6241700361315c01a58adb5a,ysyoolaw,...,None,"{'cid': '751968397.1657576250', 'ua': 'Mozilla...",main.location-lawyers,location-lawyers-all_n,None,1,normal,인천/부천,인천/부천,0983-유용수
1,62cf529497ed751822480765,1,NaN,2022-07-14 08:17:40.882,2022-07-14 08:40:26.665,2022-07-14 08:22:29,62cf5115861e5e0075d98ab7,62cf4d970ddda8001d64ef7c,559a4230131592e74cbd9cf7,sendjw,...,None,"{'cid': '2141578416.1657752904', 'ua': 'Mozill...",main.location-lawyers,location-lawyers-visiting_p,None,0,location,강동/송파,강동/송파,0208-서지원
2,62cff53097ed751822c8cd18,1,NaN,2022-07-14 19:51:28.250,2022-07-14 19:52:55.190,2022-07-14 19:51:28,62cff4a24da56100c545a60d,62b15cb5a08dea00206f3d40,53101398a52329d5060002b6,lawggeun111,...,None,"{'cid': '73382140.1655777325', 'ua': 'Mozilla/...",main.location-lawyers,location-lawyers-visiting_n,None,8,normal,종로/중구/용산,종로/중구/용산,0114-조석근
3,62c976b3506f4dec56b35e78,1,NaN,2022-07-09 21:38:11.451,2022-07-09 21:41:40.174,2022-07-09 21:38:11,62c975d275dd0d020616a32f,62c916fa5794f4010b004afb,5fc4786c43d8b200c4e5366e,ianlaw09,...,None,"{'cid': '311424555.1657345616', 'ua': 'Mozilla...",main.lawyers,search-lawyers-30visit_n,None,0,normal,대여금/채권추심,빌려준돈받는법,2782-주영재
4,62cb6f75506f4dec566c86a6,1,NaN,2022-07-11 09:31:49.541,2022-07-11 09:33:46.031,2022-07-11 09:31:49,62cb6ec5c33bd204aa1e55b4,62cb6dca0a6edd06f0b31237,605c3338acc78e00a8d89bd6,toughyjy931,...,62cb6dcb640af4058e0b6b5c,"{'cid': '587118101.1657499067', 'ua': 'Mozilla...",main.location-lawyers,location-lawyers-phone_n,None,6,normal,강남,강남,5032-윤준용
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5172,62c7cc5cb2306052f9db2fd4,1,NaN,2022-07-08 15:19:08.638,2022-07-08 15:24:26.158,2022-07-08 15:19:08,62c7cafe83b59002caf48d5a,62c3e04d97c203001e22d53a,5f66015f8c80fa00b0fec472,jinwhi0112,...,None,"{'cid': '881190208.1657004014', 'ua': 'Mozilla...",main.lawyers,search-lawyers-fifteen_p,None,13,category,대여금/채권추심,대여금/채권추심,8261-김진휘
5173,62ca88dc506f4dec561669e6,1,NaN,2022-07-10 17:07:56.162,2022-07-10 17:08:30.941,2022-07-10 17:07:55,62ca875290fc8d0206ac8409,62ca7ab46cb7df05ee26c7b2,62494576bb00d8017576aef0,lawzeus1,...,62ca7ab597445700f73bd7ab,"{'cid': '1624402573.1657436834', 'ua': 'Mozill...",main.lawyers,search-lawyers-fifteen_p,None,0,category,고소/소송절차,고소/소송절차,5327-전선재
5174,62cc0ce2506f4dec56eae185,1,NaN,2022-07-11 20:43:30.367,2022-07-11 20:43:30.367,NaT,62cc0bc35365b300d4b357e2,62cc00137c497900fc1f047a,5e6afa8917c35b02785b1569,None,...,62cc04e410f3fb00620d0c27,"{'cid': '1163295170.1657536459', 'ua': 'Mozill...",main.lawyers,search-lawyers-fifteen_p,None,5,category,마약/도박,대마초,9420-김장천
5175,62cd5570506f4dec56a437ec,1,NaN,2022-07-12 20:05:20.585,2022-07-12 20:05:56.376,2022-07-12 20:05:20,62cd5115892ef300c8d4b865,62cd4c3a8f32b50089d6fdba,5d5ce14dcd2f2438a5e051a9,prolawyer119,...,62cd4c3b2b6cfe00d79fc83f,"{'cid': '2090616946.1657617921', 'ua': 'Mozill...",main.lawyers,search-lawyers-fifteen_p,None,23,category,폭행/협박/상해 일반,폭행,9054-박성현


In [5]:
adorder_query = '''
WITH A AS (
      SELECT
      _id as adorders_id
      ,lawyer as lawyers_id
      ,status as adorders_status
      ,date(DATETIME(createdAt,'Asia/Seoul')) as adorders_createdAt_date
      ,DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', regexp_extract(term, r"'startAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})")), 'Asia/Seoul')  as adorders_startAt_timestamp
      ,DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', regexp_extract(term, r"'endAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})")), 'Asia/Seoul') as adorders_endAt_timestamp
      , adLocations
      , keywords
      , categories
      , item
    from `lawtalk-bigquery.raw.adorders`
    where date(createdAt) <= '2022-12-31'
)
, B AS (
      SELECT 
      orders_id
      ,_id as adpayments_id
    FROM `lawtalk-bigquery.raw.adpayments`, unnest(REGEXP_EXTRACT_ALL(orders, r"ObjectId\('(.*?)'\)")) as orders_id
        where date(createdAt) <= '2022-12-31'
)
, C AS (
      select
      _id as adpayments_id
      ,method as adpayments_method
      ,status as adpayments_status
    from `lawtalk-bigquery.raw.adpayments`
        where date(createdAt) <= '2022-12-31'
)
, adorders_history AS (
SELECT
    A.adorders_id
    , A.lawyers_id
    , A.adorders_createdAt_date
    , date(A.adorders_startAt_timestamp) as adorders_start_date
    , date(A.adorders_endAt_timestamp) as adorders_end_date
    , A.adorders_status
    , C.adpayments_id
    , C.adpayments_status
    , C.adpayments_method
    , A.adLocations
    , A.keywords
    , A.categories
    , A.item
FROM A 
  LEFT OUTER JOIN B
    ON A.adorders_id = B.orders_id
  LEFT OUTER JOIN C
    ON B.adpayments_id = C.adpayments_id
)
, adorders_pausehistory_tmp AS (
  SELECT 
  _id as adorders_id
  ,pauseHistory
  ,regexp_extract_all(pauseHistory, r"'startAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})") as pauseHistory_startAt
  ,regexp_extract_all(pauseHistory, r"'endAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})") as pauseHistory_endAt
FROM `lawtalk-bigquery.raw.adorders`
where 1=1
  and date(DATETIME(createdAt, 'Asia/Seoul')) <= '2022-12-31'
  and REGEXP_CONTAINS(pauseHistory, 'ObjectId')
)
, adorders_pausehistory AS (
SELECT 
  adorders_id
  ,DATE(DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', pauseHistory_startAt), 'Asia/Seoul')) as pauseHistory_startAt_date
  ,DATE(DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', pauseHistory_endAt), 'Asia/Seoul')) as pauseHistory_endAt_date
FROM adorders_pausehistory_tmp
  ,unnest(pauseHistory_startAt) as pauseHistory_startAt with offset as pos1
  ,unnest(pauseHistory_endAt) as pauseHistory_endAt with offset as pos2
where 1=1
and pos1=pos2
)
, adorder_with_pause AS (
  SELECT 
    adorders_id
    , lawyers_id
    , adorders_createdAt_date
    , adorders_start_date
    , adorders_end_date
    , pauseHistory_startAt_date
    , pauseHistory_endAt_date
    , adpayments_id
    , adorders_status
    , adpayments_status
    , adpayments_method
    , adLocations
    , keywords
    , categories
    , item
  FROM adorders_history AS A
  LEFT JOIN adorders_pausehistory AS B
  USING (adorders_id)
)
, adorders_item AS (
SELECT
  A.*
  ,B.name as item_name
  ,B.price
FROM adorder_with_pause AS A
left outer join `lawtalk-bigquery.raw.adproducts` as B on A.item = B._id
WHERE 1 = 1
)
, adorders_adloaction AS
(
  SELECT
    * EXCEPT(adLocations_id)
    , adLocations_id
  FROM adorders_item
   ,UNNEST(REGEXP_EXTRACT_ALL(adLocations, r"ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)")) AS adLocations_id
)
,adlocations_name as
(
  SELECT
    A._id,
    A.name as adlocations_name,
    A.adLocationGroup as adLocationGroup_id,
    B.name as adLocationGroup_id_name
  FROM `lawtalk-bigquery.raw.adlocations` as A
  LEFT OUTER JOIN `lawtalk-bigquery.raw.adlocationgroups` as B on A.adLocationGroup = B._id
)
, adlocation AS (
  SELECT 
    A.adorders_id
    , adlocations_name
    , adLocationGroup_id
    , adLocationGroup_id_name
  FROM adorders_adloaction AS A
  LEFT JOIN adlocations_name AS B
  ON A.adLocations_id = B._id
)
, adkeywords_unnest AS (
    SELECT 
    _id as adkeywords_id
    ,name as adkeywords_name
    , adCategories_id
  FROM `lawtalk-bigquery.raw.adkeywords` AS adkeywords
    ,UNNEST(REGEXP_EXTRACT_ALL(adCategories, r"ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)")) as adCategories_id  
)
, adkeywords_with_category AS (
  SELECT
    adcategories.name AS adCategories_name
    , adkeywords.adCategories_id
    , adkeywords.adkeywords_name
    , adkeywords.adkeywords_id
  FROM adkeywords_unnest AS adkeywords
  LEFT JOIN `lawtalk-bigquery.raw.adcategories` as adcategories on adkeywords.adCategories_id = adcategories._id
)
, adorders_keywords_tmp as
(
  SELECT
    * EXCEPT(keywords_id)
    , keywords_id
  FROM adorders_item
    ,UNNEST(regexp_extract_all(keywords, r"ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)")) AS keywords_id
)
, adkeyword AS (
  SELECT
    A.adorders_id
    , B.*
  FROM adorders_keywords_tmp AS A
  LEFT JOIN adkeywords_with_category AS B
  ON A.keywords_id = B.adkeywords_id 
)
, adorders_item_adCategory_tmp AS (
  SELECT  
    * EXCEPT(adCategoryId)
    , adCategoryId
  FROM adorders_item
  , UNNEST(regexp_extract_all(categories, r"'adCategoryId': ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)"))  as adCategoryId
)
, adnormal AS (
SELECT
  A.adorders_id
  ,C.name as adCategory_name
FROM adorders_item_adCategory_tmp AS A
left outer join `lawtalk-bigquery.raw.adcategories` as C on A.adCategoryId = C._id
WHERE 1 = 1
)
, adorder_end AS (
SELECT * FROM adorders_item
LEFT JOIN adlocation USING (adorders_id)
LEFT JOIN adkeyword USING (adorders_id)
LEFT JOIN adnormal USING (adorders_id)
)
, slug_lawyers AS (
SELECT 
  _id
  , slug
FROM `lawtalk-bigquery.raw.lawyers` AS B
WHERE 1 = 1
  AND REGEXP_CONTAINS(slug, r'\d{4}-[가-힣]+')
)
, adorder_lawyer AS (
SELECT 
  B._id AS lawyers_id
  , B.slug
  , A.item_name
  , A.adorders_start_date	
  , A.adorders_end_date	
  , A.pauseHistory_startAt_date	
  , A.pauseHistory_endAt_date
  , A.adorders_status
  , A.adpayments_status
FROM slug_lawyers AS B
FULL OUTER JOIN adorder_end AS A
ON A.lawyers_id = B._id
)
, tmp_table AS (
SELECT 
  slug
  , lawyers_id
  , item_name
  , adorders_start_date	
  , adorders_end_date	
  , pauseHistory_startAt_date	
  , pauseHistory_endAt_date
  , adorders_status
  , adpayments_status
  , CASE WHEN adorders_status = 'apply' AND adpayments_status = 'paid' AND '2022-07-01' BETWEEN adorders_start_date AND adorders_end_date AND ('2022-07-01' NOT BETWEEN pauseHistory_startAt_date AND pauseHistory_endAt_date	OR pauseHistory_startAt_date IS NULL) THEN '광고주' ELSE '비광고주' END AS ad_tf
  , CASE WHEN adorders_status = 'apply' AND adpayments_status = 'paid' AND '2022-07-01' BETWEEN adorders_start_date AND adorders_end_date AND ('2022-07-01' NOT BETWEEN pauseHistory_startAt_date AND pauseHistory_endAt_date	OR pauseHistory_startAt_date IS NULL) THEN 1 ELSE 0 END AS ad_cnt
FROM adorder_lawyer
WHERE 1 = 1
  -- AND '2022-07-01' BETWEEN adorders_start_date AND adorders_end_date
  -- AND pauseHistory_startAt_date IS NOT NULL
  -- AND ('2022-07-01' NOT BETWEEN pauseHistory_startAt_date AND pauseHistory_endAt_date	OR pauseHistory_startAt_date IS NULL)
)
SELECT 
  slug
  , lawyers_id
  , STRING_AGG(ad_tf ORDER BY ad_tf LIMIT 1) AS ad_tf
  , SUM(ad_cnt) AS ad_cnt
FROM tmp_table
GROUP BY 1, 2
ORDER BY 3 DESC
;
'''

In [ ]:
adorder_query = '''
WITH A AS (
      SELECT
      _id as adorders_id
      ,lawyer as lawyers_id
      ,status as adorders_status
      ,date(DATETIME(createdAt,'Asia/Seoul')) as adorders_createdAt_date
      ,DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', regexp_extract(term, r"'startAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})")), 'Asia/Seoul')  as adorders_startAt_timestamp
      ,DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', regexp_extract(term, r"'endAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})")), 'Asia/Seoul') as adorders_endAt_timestamp
      , adLocations
      , keywords
      , categories
      , item
    from `lawtalk-bigquery.raw.adorders`
    where date(createdAt) <= '2022-12-31'
)
, B AS (
      SELECT 
      orders_id
      ,_id as adpayments_id
    FROM `lawtalk-bigquery.raw.adpayments`, unnest(REGEXP_EXTRACT_ALL(orders, r"ObjectId\('(.*?)'\)")) as orders_id
        where date(createdAt) <= '2022-12-31'
)
, C AS (
      select
      _id as adpayments_id
      ,method as adpayments_method
      ,status as adpayments_status
    from `lawtalk-bigquery.raw.adpayments`
        where date(createdAt) <= '2022-12-31'
)
, adorders_history AS (
SELECT
    A.adorders_id
    , A.lawyers_id
    , A.adorders_createdAt_date
    , date(A.adorders_startAt_timestamp) as adorders_start_date
    , date(A.adorders_endAt_timestamp) as adorders_end_date
    , A.adorders_status
    , C.adpayments_id
    , C.adpayments_status
    , C.adpayments_method
    , A.adLocations
    , A.keywords
    , A.categories
    , A.item
FROM A 
  LEFT OUTER JOIN B
    ON A.adorders_id = B.orders_id
  LEFT OUTER JOIN C
    ON B.adpayments_id = C.adpayments_id
)
, adorders_pausehistory_tmp AS (
  SELECT 
  _id as adorders_id
  ,pauseHistory
  ,regexp_extract_all(pauseHistory, r"'startAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})") as pauseHistory_startAt
  ,regexp_extract_all(pauseHistory, r"'endAt': datetime.datetime\((\d{4}, \d{1,2}, \d{1,2}, \d{1,2}, \d{1,2})") as pauseHistory_endAt
FROM `lawtalk-bigquery.raw.adorders`
where 1=1
  and date(DATETIME(createdAt, 'Asia/Seoul')) <= '2022-12-31'
  and REGEXP_CONTAINS(pauseHistory, 'ObjectId')
)
, adorders_pausehistory AS (
SELECT 
  adorders_id
  ,DATE(DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', pauseHistory_startAt), 'Asia/Seoul')) as pauseHistory_startAt_date
  ,DATE(DATETIME(parse_timestamp('%Y, %m, %e, %H, %M', pauseHistory_endAt), 'Asia/Seoul')) as pauseHistory_endAt_date
FROM adorders_pausehistory_tmp
  ,unnest(pauseHistory_startAt) as pauseHistory_startAt with offset as pos1
  ,unnest(pauseHistory_endAt) as pauseHistory_endAt with offset as pos2
where 1=1
and pos1=pos2
)
, adorder_with_pause AS (
  SELECT 
    adorders_id
    , lawyers_id
    , adorders_createdAt_date
    , adorders_start_date
    , adorders_end_date
    , pauseHistory_startAt_date
    , pauseHistory_endAt_date
    , adpayments_id
    , adorders_status
    , adpayments_status
    , adpayments_method
    , adLocations
    , keywords
    , categories
    , item
  FROM adorders_history AS A
  LEFT JOIN adorders_pausehistory AS B
  USING (adorders_id)
)
, adorders_item AS (
SELECT
  A.*
  ,B.name as item_name
  ,B.price
FROM adorder_with_pause AS A
left outer join `lawtalk-bigquery.raw.adproducts` as B on A.item = B._id
WHERE 1 = 1
)
, adorders_adloaction AS
(
  SELECT
    * EXCEPT(adLocations_id)
    , adLocations_id
  FROM adorders_item
   ,UNNEST(REGEXP_EXTRACT_ALL(adLocations, r"ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)")) AS adLocations_id
)
,adlocations_name as
(
  SELECT
    A._id,
    A.name as adlocations_name,
    A.adLocationGroup as adLocationGroup_id,
    B.name as adLocationGroup_id_name
  FROM `lawtalk-bigquery.raw.adlocations` as A
  LEFT OUTER JOIN `lawtalk-bigquery.raw.adlocationgroups` as B on A.adLocationGroup = B._id
)
, adlocation AS (
  SELECT 
    A.adorders_id
    , adlocations_name
    , adLocationGroup_id
    , adLocationGroup_id_name
  FROM adorders_adloaction AS A
  LEFT JOIN adlocations_name AS B
  ON A.adLocations_id = B._id
)
, adkeywords_unnest AS (
    SELECT 
    _id as adkeywords_id
    ,name as adkeywords_name
    , adCategories_id
  FROM `lawtalk-bigquery.raw.adkeywords` AS adkeywords
    ,UNNEST(REGEXP_EXTRACT_ALL(adCategories, r"ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)")) as adCategories_id  
)
, adkeywords_with_category AS (
  SELECT
    adcategories.name AS adCategories_name
    , adkeywords.adCategories_id
    , adkeywords.adkeywords_name
    , adkeywords.adkeywords_id
  FROM adkeywords_unnest AS adkeywords
  LEFT JOIN `lawtalk-bigquery.raw.adcategories` as adcategories on adkeywords.adCategories_id = adcategories._id
)
, adorders_keywords_tmp as
(
  SELECT
    * EXCEPT(keywords_id)
    , keywords_id
  FROM adorders_item
    ,UNNEST(regexp_extract_all(keywords, r"ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)")) AS keywords_id
)
, adkeyword AS (
  SELECT
    A.adorders_id
    , B.*
  FROM adorders_keywords_tmp AS A
  LEFT JOIN adkeywords_with_category AS B
  ON A.keywords_id = B.adkeywords_id 
)
, adorders_item_adCategory_tmp AS (
  SELECT  
    * EXCEPT(adCategoryId)
    , adCategoryId
  FROM adorders_item
  , UNNEST(regexp_extract_all(categories, r"'adCategoryId': ObjectId\(\'([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)\'\)"))  as adCategoryId
)
, adnormal AS (
SELECT
  A.adorders_id
  ,C.name as adCategory_name
FROM adorders_item_adCategory_tmp AS A
left outer join `lawtalk-bigquery.raw.adcategories` as C on A.adCategoryId = C._id
WHERE 1 = 1
)
, adorder_end AS (
SELECT * FROM adorders_item
LEFT JOIN adlocation USING (adorders_id)
LEFT JOIN adkeyword USING (adorders_id)
LEFT JOIN adnormal USING (adorders_id)
)
, slug_lawyers AS (
SELECT 
  _id
  , slug
FROM `lawtalk-bigquery.raw.lawyers` AS B
WHERE 1 = 1
  AND REGEXP_CONTAINS(slug, r'\d{4}-[가-힣]+')
)
, adorder_lawyer AS (
SELECT 
  B._id AS lawyers_id
  , B.slug
  , A.item_name
  , A.adorders_start_date	
  , A.adorders_end_date	
  , A.pauseHistory_startAt_date	
  , A.pauseHistory_endAt_date
  , A.adorders_status
  , A.adpayments_status
FROM slug_lawyers AS B
FULL OUTER JOIN adorder_end AS A
ON A.lawyers_id = B._id
)
, tmp_table AS (
SELECT 
  slug
  , lawyers_id
  , item_name
  , adorders_start_date	
  , adorders_end_date	
  , pauseHistory_startAt_date	
  , pauseHistory_endAt_date
  , adorders_status
  , adpayments_status
  , CASE WHEN adorders_status = 'apply' AND adpayments_status = 'paid' AND '2022-07-01' BETWEEN adorders_start_date AND adorders_end_date AND ('2022-07-01' NOT BETWEEN pauseHistory_startAt_date AND pauseHistory_endAt_date	OR pauseHistory_startAt_date IS NULL) THEN '광고주' ELSE '비광고주' END AS ad_tf
  , CASE WHEN adorders_status = 'apply' AND adpayments_status = 'paid' AND '2022-07-01' BETWEEN adorders_start_date AND adorders_end_date AND ('2022-07-01' NOT BETWEEN pauseHistory_startAt_date AND pauseHistory_endAt_date	OR pauseHistory_startAt_date IS NULL) THEN 1 ELSE 0 END AS ad_cnt
FROM adorder_lawyer
WHERE 1 = 1
  -- AND '2022-07-01' BETWEEN adorders_start_date AND adorders_end_date
  -- AND pauseHistory_startAt_date IS NOT NULL
  -- AND ('2022-07-01' NOT BETWEEN pauseHistory_startAt_date AND pauseHistory_endAt_date	OR pauseHistory_startAt_date IS NULL)
)
SELECT 
  slug
  , lawyers_id
  , STRING_AGG(ad_tf ORDER BY ad_tf LIMIT 1) AS ad_tf
  , SUM(ad_cnt) AS ad_cnt
FROM tmp_table
GROUP BY 1, 2
ORDER BY 3 DESC
;
'''

advice_query = '''
WITH advice AS (
SELECT
  _id
  , lawyer AS lawyers_id
  , adCategory
  , createdAt
  , status
FROM `lawtalk-bigquery.raw.advice`
WHERE 1 = 1 
  AND status = 'complete'
  AND createdAt between '2022-07-01' AND '2022-07-15'
)
, adCategory AS (
SELECT
  _id AS adCategory
  , name AS adCategory_name
FROM `lawtalk-bigquery.raw.adcategories`
)
SELECT 
  _id
  , lawyers_id
  , adCategory
  , createdAt
  , status
  , adCategory_name
FROM advice A
LEFT JOIN adCategory B
USING (adCategory)
'''


advicet_query = '''
WITH advicetransaction_tmp AS (
SELECT
  _id,
  row_number() over(PARTITION BY advice ORDER BY updatedAt DESC) AS no_by_advice,
  number, 
  DATETIME(createdAt, 'Asia/Seoul') as createdAt,
  DATETIME(updatedAt, 'Asia/Seoul') as updatedAt,
  DATETIME(requestedAt, 'Asia/Seoul') as requestedAt,
  advice,
  user,
  lawyer AS lawyer_id,
  seller,
  tid,
  status,
  method,
  originFee,
  canceled,
  coupon,
  metadata,
  REGEXP_EXTRACT(metadata, r"'page': '([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)'") as metadata_page,
  REGEXP_EXTRACT(metadata, r"'source': '([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)'") as metadata_source,
  REGEXP_EXTRACT(metadata, r"'element': '([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)'") as metadata_element,
  REGEXP_EXTRACT(metadata, r"'position': ([0-9]+),") as metadata_position,
  REGEXP_EXTRACT(metadata, r"'context': '([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)'") as metadata_context,
  REGEXP_EXTRACT(metadata, r"'contextAdditional': '([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)'") as metadata_contextAdditional,
  REGEXP_EXTRACT(metadata, r"'extraInfo': '([ㄱ-ㅎ가-힣a-zA-Z0-9 !#$%&()*+,-./:;<=>?@[\]^_`{|}~]+)'") as metadata_extraInfo,
FROM `lawtalk-bigquery.raw.advicetransactions`
where 1=1
  AND DATE(DATETIME(createdAt, 'Asia/Seoul')) BETWEEN '2022-07-01' AND '2022-07-15'
  AND status = 'paid'
)
, slug_lawyers AS (
SELECT 
  _id
  , slug
FROM `lawtalk-bigquery.raw.lawyers` AS B
WHERE 1 = 1
  AND REGEXP_CONTAINS(slug, r'\d{4}-[가-힣]+')
)
SELECT 
    A.*
    , B.slug
FROM advicetransaction_tmp AS A
LEFT JOIN slug_lawyers AS B
ON A.lawyer_id = B._id
WHERE 1 = 1
'''

ga_query = '''
SELECT 
	event_date	
	, event_timestamp
	, user_pseudo_id
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'lawyer_id') as slug 
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'search_lawyer_노출이유_단어') as expose_reason
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'search_keyword') as search_keyword
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'ga_session_id') as ga_session_id 
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'page_title') as page_title 
    , (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'page_location') as page_location 
FROM `lawtalk-bigquery.analytics_265523655.events_*`
WHERE 1 = 1 
	AND _TABLE_SUFFIX BETWEEN '20220701' AND '20220715'
	AND event_name = 'tag.search.프로필노출'
;
'''

gae_query = '''
SELECT 
	event_date	
	, event_timestamp
	, user_pseudo_id
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'keyword') as keyword
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'ga_session_id') as ga_session_id 
	, (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'page_title') as page_title 
  , (SELECT param.value.string_value FROM UNNEST(event_params) as param WHERE param.key = 'page_location') as page_location 
FROM `lawtalk-bigquery.analytics_265523655.events_*`
WHERE 1 = 1 
	AND _TABLE_SUFFIX BETWEEN '20220701' AND '20220715'
	AND event_name = 'tag.상단검색창직접입력'
'''

In [ ]:
adorder = bigquery_to_pandas(adorder_query)
advice = bigquery_to_pandas(advice_query)
advicet = bigquery_to_pandas(advicet_query)
ga = bigquery_to_pandas(ga_query)
gae = bigquery_to_pandas(gae_query)
lawyer = bigquery_to_pandas(lawyer_query)

In [ ]:
keyword = gs_append(url = 'https://docs.google.com/spreadsheets/d/1n9YKC3L9_d4QRKjlDqbHelxw5nC3Fz8hrgbHahOHhxM/edit#gid=0')
keyword.columns = ["type1", "type2", "search_keyword"]

In [ ]:
def get_gini(df, val_col, color = "red", plot_tf = True) :
    tmp = df.copy()
    tmp["ratio"] = tmp[val_col] / tmp[val_col].max()
    tmp["rnk"] = tmp.ratio.rank(method = "first")  / tmp.shape[0]
    tmp = tmp.sort_values("rnk")
    tmp["eq_line"] = [i / tmp.shape[0] for i in range(tmp.shape[0])]
    
    lorenz = (tmp.rnk * tmp.shape[0] * (tmp.ratio / tmp.shape[0])).sum()
    tri = ((tmp.rnk * tmp.shape[0]).max() * tmp.rnk.max()) / 2
    
    if plot_tf == True :
        print((
            ggplot(data = tmp) +
            geom_col(mapping = aes(x = "rnk", y = "ratio"), fill = color, alpha = 0.6) +
            geom_line(mapping = aes(x = "rnk", y = "eq_line"), group = 1) +
            theme_bw() +
            theme(figure_size = (10, 7), text = element_text(angle = 90, fontproperties = font), axis_title=element_blank())
        ))
    
    return (tri - lorenz) / tri

In [ ]:
ga_ad = pd.merge(ga, adorder[["slug", "ad_tf", "ad_cnt"]], on = "slug", how = "left")
ga_ad.loc[ga_ad.search_keyword.isin(keyword.search_keyword), "mapping"] = 1
ga_ad.loc[~ga_ad.search_keyword.isin(keyword.search_keyword), "mapping"] = 0

ga_ad.search_keyword = ga_ad.search_keyword.str.replace(pat = "~2F", repl = "/")
ga_ad.search_keyword = ga_ad.search_keyword.fillna("")
ga_ad.space_len = ga_ad.search_keyword.str.split(" |/", regex = True).apply(lambda x : len(x))

if1 = (ga_ad.search_keyword == ga_ad.expose_reason) & (ga_ad.search_keyword.notna()) & (ga_ad.search_keyword != "") &  (~ga_ad.search_keyword.str.contains("/")) & (~ga_ad.search_keyword.isin(lawyer.name))

if1_ = (ga_ad.search_keyword != ga_ad.expose_reason) & (ga_ad.search_keyword.notna()) & (ga_ad.search_keyword != "") &  (~ga_ad.search_keyword.str.contains("/")) & (~ga_ad.search_keyword.isin(lawyer.name))

In [ ]:
# 조건1 : 검색어와 검색 결과가 같지 않고(대분야 클릭으로 들어오지 않은) 변호사 이름이 아닌 검색어들
if1 = (ga_ad.search_keyword != ga_ad.expose_reason) & (ga_ad.search_keyword.notna()) & (ga_ad.search_keyword != "") &  (~ga_ad.search_keyword.str.contains("/")) & (~ga_ad.search_keyword.isin(lawyer.name))
# 조건2 : 조건1 + 한단어(뛰어쓰기가 없도록)
if2 = (ga_ad.search_keyword != ga_ad.expose_reason) & (ga_ad.search_keyword.notna()) & (ga_ad.search_keyword != "") & (ga_ad.space_len == 1) & (~ga_ad.search_keyword.str.contains("/")) & (~ga_ad.search_keyword.isin(lawyer.name))
# 조건3 : 조건1에 100개 이상 검색된 결과
if3 = (ga_ad.search_keyword != ga.expose_reason) & (ga_ad.search_keyword.notna()) & (ga_ad.search_keyword != "") &  (~ga_ad.search_keyword.str.contains("/")) & (~ga_ad.search_keyword.isin(lawyer.name))
# 조건4 : 조건2에 100개 이상 검색된 결과
if4 = (ga_ad.search_keyword != ga_ad.expose_reason) & (ga_ad.search_keyword.notna()) & (ga_ad.search_keyword != "") & (ga_ad.space_len == 1) & (~ga_ad.search_keyword.str.contains("/")) & (~ga_ad.search_keyword.isin(lawyer.name))

In [ ]:
ga_ad[if1].groupby(["search_keyword"]).slug.count().sort_values(ascending = False)
ga_ad[if2].groupby("search_keyword").slug.count().sort_values(ascending = False)
ga_ad[if3].groupby("search_keyword").slug.count().query("search_keyword > 100").sort_values(ascending = False)
ga_ad[if4].groupby("search_keyword").slug.count().query("search_keyword > 100").sort_values(ascending = False)

ga_ad[if1].groupby("mapping").user_pseudo_id.nunique()

ga_ad[if1 & (ga_ad.mapping == 1)].groupby("search_keyword").user_pseudo_id.nunique().sort_values(ascending = False)
ga_ad[if1 & (ga_ad.mapping == 0)].groupby("search_keyword").user_pseudo_id.nunique().sort_values(ascending = False)


ga[if1].groupby("search_keyword").slug.nunique().sort_values(ascending = False)
ga[if1].groupby("search_keyword").slug.count().sort_values(ascending = False)

ga[if1].search_keyword.value_counts().reset_index().search_keyword.sum()
ga[if2].search_keyword.value_counts().reset_index().search_keyword.sum()
ga[if3].search_keyword.value_counts().reset_index().query("search_keyword > 100").search_keyword.sum()
ga[if4].search_keyword.value_counts().reset_index().query("search_keyword > 100").search_keyword.sum()



get_gini(ga[if1].groupby("search_keyword").slug.nunique().reset_index(), "slug")
get_gini(ga[if1].groupby("search_keyword").slug.count().reset_index(), "slug")

get_gini(ga[if2].groupby("search_keyword").slug.nunique().reset_index(), "slug")

get_gini(ga[if2].search_keyword.value_counts().reset_index(), "search_keyword")
get_gini(ga[if3].search_keyword.value_counts().reset_index().query("search_keyword > 100"), "search_keyword")
get_gini(ga[if4].search_keyword.value_counts().reset_index().query("search_keyword > 100"), "search_keyword")

get_gini(ga.groupby("lawyer_id").event_date.count().reset_index(), "event_date")
get_gini(ga.groupby("slug").user_pseudo_id.nunique().reset_index(), "user_pseudo_id")

ga[if1].groupby("slug").event_date.count().sort_values(ascending = False)

get_gini(ga[(if1)].groupby("lawyer_id").event_date.count().reset_index(), "event_date")
get_gini(ga[if2].groupby("lawyer_id").event_date.count().reset_index(), "event_date")

get_gini(ga[(if1)].groupby("slug").event_date.count().reset_index(), "event_date")
get_gini(ga[if2].groupby("slug").event_date.count().reset_index(), "event_date")

In [ ]:
''''''
if1 = (advicet.metadata_contextAdditional != advicet.metadata_extraInfo) & (~advicet.metadata_extraInfo.isin(lawyer.name)) & (~advicet.metadata_extraInfo.fillna("").str.contains("/"))

advicet.metadata_context.value_counts()


# 5177 -> 1550 약 30%
advicet[(advicet.metadata_context == 'category') & (advicet.metadata_extraInfo != advicet.metadata_contextAdditional)].metadata_contextAdditional.value_counts()[:10]

for i in advicet[(advicet.metadata_context == 'category') & (advicet.metadata_extraInfo != advicet.metadata_contextAdditional)].metadata_contextAdditional.value_counts()[:10].index :
    print(advicet[advicet.metadata_contextAdditional.fillna("").str.contains(i)].shape[0])

'''
'''


get_gini(advicet[advicet.metadata_contextAdditional.fillna("").str.contains('임대차')].groupby("lawyer_id")._id.count().reset_index(), "_id")

advicet[advicet.metadata_contextAdditional.fillna("").str.contains('폭행/협박/상해 일반')]
advicet[advicet.metadata_contextAdditional.fillna("").str.contains('임대차')]

advicet[if1]
advicet[if1 & (advicet.metadata_extraInfo.isin(keyword.search_keyword))]

advicet[if1].metadata_extraInfo.value_counts()[:100]
advicet[advicet.metadata_extraInfo == "통신매체이용음란죄"].metadata_contextAdditional.value_counts()

advice


for i in list(advicet[if1].metadata_extraInfo.value_counts()[:10].index) :
    print(advicet[advicet.metadata_extraInfo == i].groupby("slug")._id.count().sort_values(ascending = False)[:1])    
    get_gini(advicet[advicet.metadata_extraInfo == i].groupby("slug")._id.count().reset_index(), "_id")